In [440]:
import connect

In [441]:
postgres = connect.PostgreSQLConnection("login_postgres.json")

In [442]:
postgres.cursor.execute('''SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';''')
tables = [i[1] for i in postgres.cursor.fetchall()]
tables

['test', 'test_1']

In [443]:
import pandas as pd

In [444]:
postgres.cursor.execute(f'''SELECT * FROM {tables[0]};''')
data0 = pd.DataFrame(data=postgres.cursor.fetchall(),columns=[c[0] for c in postgres.cursor.description])
data0.to_csv("test_0.csv")

In [445]:
postgres.cursor.execute(f'''SELECT * FROM {tables[1]};''')
data1 = pd.DataFrame(data=postgres.cursor.fetchall(),columns=[c[0] for c in postgres.cursor.description])
data1.to_csv("test_1.csv")

In [446]:
data0.reset_index()
data0.drop("id",axis=1,inplace=True)
data0

,name,surname
0,CllUBphNKkwrdCkRZjiG,PlxWrLbWYNFvgxsELbuK
1,clxYppuiQsRwkMeNVzTj,PfkoItYQZmMgKcITsHSD
2,LXCIEaYcSdmlOIeWQQtY,ReFNpMNsKNCAHgzHptNs
3,cXUnwIeZHwXbzKVOLiKE,awWryLoZzlInMgCPCWqi
4,LLTGGVTYFeqfKvNEQAwl,vDATKstnzJdyQFGRCjSi
...,...,...
95,ccYTzLcqbUnJBpTTRDqQ,qaixzcXKySWqESakBcLu
96,wzGiHnFVdcyqRKyhjhTD,qfOIizIGsHVRslmHYMEm
97,gaggXVbxvKYjHDbfQmYY,oteduczOIlMwtfLpvuho
98,fdDJfArDuKpniqKlYEfy,NMvIYuLvrQyBBsCUWJve


In [447]:
import pandas_decimal
data1.reset_index()
data1.drop("id",axis=1,inplace=True)

In [448]:
data1["salary"] = data1.salary.apply(lambda x: abs(x)).astype("decimal[2]")
data1

,salary,city,country
0,1061.56,NzRMVaONPdQXgoqdCUeq,MuBviFjvjHslOlPTXZfr
1,8749.45,YZEwjweOVYisDNjnVphZ,wCnuRLaJgxWTNeKNOVBd
2,1919.24,uTImtHBSkGZlCHmUJyXI,ZhryLqcHkVoqwjCQBRlS
3,2355.19,gbvwpzInpwiZXQudYmnc,wAMTzCmwBhSxXxgGGWVY
4,9426.76,uGuqVEnJRyTwdzdIsXJO,oHppmsaVQaiAVAVxpNkk
...,...,...,...
95,2767.67,HJGMWNbLNDrjXCrummSG,rTJPkbYGSEOvxqlPSfXh
96,8494.42,KMYwuATqfdsoyaegDPxV,ojEcXpXkdvxwzOBTvfdJ
97,6527.85,QiLrdbFEmWhxuGeLMZpx,NpmoOBaXPVAVIBjxnxwq
98,5236.09,RYtVewZccabhiKEJzmoZ,ZUukWDKWsNDCSchekHFF


In [449]:
data = pd.concat([data0,data1],axis=1)
mysql = connect.MySQLConnection("login_mysql.json")

In [450]:
data["city"] = data["city"].str.capitalize().str[0:8:2]
data.country = data.country.str.capitalize().str[0:5]
data.head(5)

,name,surname,salary,city,country
0,CllUBphNKkwrdCkRZjiG,PlxWrLbWYNFvgxsELbuK,1061.56,Nrvo,Mubvi
1,clxYppuiQsRwkMeNVzTj,PfkoItYQZmMgKcITsHSD,8749.45,Yeje,Wcnur
2,LXCIEaYcSdmlOIeWQQtY,ReFNpMNsKNCAHgzHptNs,1919.24,Uitb,Zhryl
3,cXUnwIeZHwXbzKVOLiKE,awWryLoZzlInMgCPCWqi,2355.19,Gvpi,Wamtz
4,LLTGGVTYFeqfKvNEQAwl,vDATKstnzJdyQFGRCjSi,9426.76,Uuvn,Ohppm


In [451]:
from sqlalchemy import create_engine
engine = create_engine("mysql://{user}:{pw}@{host}/{db}".format(host=mysql.access_data["host"], db=mysql.access_data["database"], user=mysql.access_data["user"], pw=mysql.access_data["password"]))
data.to_sql('table1', engine, if_exists='replace', index=True)

100